<a href="https://colab.research.google.com/github/haluowan/pytorch/blob/master/time_seris_prediction_in_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
% matplotlib inline

In [0]:
# hyper parameters
num_time_steps = 50
input_size = 1
hidden_size = 16
output_size = 1
lr=0.01


In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        
        self.rnn = nn.RNN(input_size = input_size,
                         hidden_size = hidden_size,
                         num_layers = 1,
                         batch_first = True)
        for p in self.rnn.parameters():
            nn.init.normal_(p,mean=0.0,std=0.001)
            
        self.linear = nn.Linear(hidden_size,output_size)
        
    def forward(self,x,hidden_prev):
        out,hidden_prev = self.rnn(x,hidden_prev)
        # [b,seq,h]
        out = out.view(-1,hidden_size)
        out = self.linear(out)
        out = out.unsqueeze(dim=0)
        return out,hidden_prev
    
        

In [0]:
model = Net()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr=lr)

In [0]:
hidden_prev = torch.zeros(1,1,hidden_size)

In [0]:
for step in range(3000):
    start = np.random.randint(3,size=1)[0]
    time_steps = np.linspace(start,start+10,num_time_steps)
    data = np.sin(time_steps)
    data = data.reshape(num_time_steps,1)
    x = torch.tensor(data[:-1]).float().view(1,num_time_steps-1,1)
    y = torch.tensor(data[1:]).float().view(1,num_time_steps-1,1)
    
    output,hidden_prev = model(x,hidden_prev)
    hidden_prev = hidden_prev.detach()
    
    loss = criterion(output,y)
    model.zero_grad()
    loss.backward()
    optimizer.step()
    
    if step % 100 == 0:
        print('Iteration:{} | Loss:{}'.format(step,loss.item()))
        


In [0]:
start = np.random.randint(3,size=1)[0]
time_steps = np.linspace(start,start + 10,num_time_steps)
data = np.sin(time_steps)
data = data.reshape(num_time_steps,1)
x = torch.tensor(data[:-1]).float().view(1,num_time_steps - 1,1)
y = torch.tensor(data[1:]).float().view(1,num_time_steps -1,1)


In [0]:
predictions = []
input = x[:,0,:]
for _ in range(x.shape[1]):
    input = input.view(1,1,1)
    (pred,hidden_prev) = model(input,hidden_prev)
    input = pred
    predictions.append(pred.detach().numpy().ravel()[0])
    
x = x.data.numpy().ravel()
y = y.data.numpy()

plt.scatter(time_steps[:-1],x.ravel(),s=90)
plt.plot(time_steps[:-1],x.ravel())

plt.scatter(time_steps[1:],predictions)
plt.show()